 # Night glucose, nutrition and lifestyle
    
 Looking for associations between lifestyle data, nutrition and night glucose levels

In [1]:
import pandas as pd

In [2]:
import LabData
from LabData.DataLoaders.Loader import Loader

In [3]:
cgmLoader = Loader.get_loader("CGMLoader")

In [4]:
cgm = cgmLoader.get_data(study_ids=3)
#cgm = cgmLoader.get_data()

In [5]:
cgm_df = cgm.df
cgm_meta = cgm.df_metadata

In [6]:
cgm_df

GlucoseValue     PPGR
RegistrationCode ConnectionID Date                                            
111527           1926         2017-11-06 14:16:00+02:00          79.0  109.750
                              2017-11-06 14:31:00+02:00          83.0  132.500
                              2017-11-06 14:46:00+02:00          86.0  153.500
                              2017-11-06 15:01:00+02:00          89.0  170.750
                              2017-11-06 15:16:00+02:00         118.0  127.375
...                                                               ...      ...
997735           5832         2019-07-21 07:45:00+03:00         117.0      NaN
                              2019-07-21 08:00:00+03:00         119.0      NaN
                              2019-07-21 08:15:00+03:00         127.0      NaN
                              2019-07-21 08:30:00+03:00         138.0      NaN
                              2019-07-21 08:45:00+03:00         135.0      NaN

[3478385 rows x 2 columns]

In [9]:
cgm_df.loc[('997735', slice(None), slice(None)), :]

GlucoseValue  PPGR
RegistrationCode ConnectionID Date                                         
997735           4596         2018-12-31 13:12:00+02:00         107.0   0.0
                              2018-12-31 13:27:00+02:00         108.0   0.0
                              2018-12-31 13:42:00+02:00         106.0   0.0
                              2018-12-31 13:57:00+02:00         102.0   0.0
                              2018-12-31 14:12:00+02:00          96.0   0.0
...                                                               ...   ...
                 5832         2019-07-21 07:45:00+03:00         117.0   NaN
                              2019-07-21 08:00:00+03:00         119.0   NaN
                              2019-07-21 08:15:00+03:00         127.0   NaN
                              2019-07-21 08:30:00+03:00         138.0   NaN
                              2019-07-21 08:45:00+03:00         135.0   NaN

[15241 rows x 2 columns]

In [10]:
def filter_by_time(df, start, end):
    
    """
    The function filters the cgm entries between certain hours of the day. 
    The output is a dataframe containing the entries between start and end hour of each day.
    :param df: CGM DataFrame
    :param start: string of a type 'hh:mm'
    :param end: string of a type 'hh:mm'
    :return: filtered DataFrame with an index set to 'RegistrationCode', 'Day' and 'Time'
    """

    df = df.reset_index(['RegistrationCode', 'ConnectionID'])
    filtered_df = df.between_time(start, end)
    pd.options.mode.chained_assignment = None  # to avoid the warning message about chained assignments
    filtered_df['Day'] = filtered_df.index.date
    filtered_df['Time'] = filtered_df.index - filtered_df.index.normalize()
    filtered_df = filtered_df.set_index(['RegistrationCode', 'Day', 'Time'])
    filtered_df = filtered_df.sort_index(level=['RegistrationCode', 'Day', 'Time'])
    
    return filtered_df

In [11]:
filter_by_time(cgm_df, '1:00', '10:00')

ConnectionID  GlucoseValue   PPGR
RegistrationCode Day        Time                                      
111527           2017-11-07 01:00:00         1926          97.0   0.00
                            01:15:00         1926          99.0   0.00
                            01:30:00         1926          94.0   0.00
                            01:45:00         1926          82.0  16.50
                            02:00:00         1926          89.0   2.25
...                                           ...           ...    ...
997735           2019-07-21 07:45:00         5832         117.0    NaN
                            08:00:00         5832         119.0    NaN
                            08:15:00         5832         127.0    NaN
                            08:30:00         5832         138.0    NaN
                            08:45:00         5832         135.0    NaN

[1331383 rows x 3 columns]

## Sleep data

In [12]:
sleep_df = pd.read_pickle('/net/mraid08/export/genie/LabData/Data/WIS/sleep_wakeup.dat')

/usr/wisdom/python3.5.3/lib/python3.5/site-packages/pandas/core/indexes/base.py:183: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)


In [13]:
sleep_df.reset_index('RegistrationCode', inplace=True)

In [14]:
sleep_df.loc[:, ['RegistrationCode']] = sleep_df.loc[:,'RegistrationCode'].astype(int).astype(str)

In [15]:
sleep_df

RegistrationCode Quality          sleep_time         wakeup_time
RunningIndex                                                                 
0                           0     NaT 2015-06-11 09:23:15 2015-06-11 09:23:21
1                           0     NaT 2015-06-11 09:56:54 2015-06-11 19:56:37
2                           0     NaT 2015-06-12 01:50:00 2015-06-12 02:29:19
3                           0     NaT 2015-06-12 02:29:09 2015-06-12 02:29:19
4                           0     NaT 2015-06-14 01:45:40 2015-06-14 11:08:19
...                       ...     ...                 ...                 ...
49756              2147483647     NaT 2015-05-22 13:32:28 2015-05-23 07:10:41
49757              2147483647     NaT 2015-05-23 01:30:34 2015-05-23 07:10:41
49758              2147483647     NaT 2015-05-24 00:24:00 2015-05-25 11:00:43
49759              2147483647     NaT 2015-05-25 00:19:00 2015-05-25 11:00:43
49760              2147483647     NaT 2015-05-27 01:40:48 2015-05-27 07:30:47

[49761 rows x 4 columns]

## CGM Metrics

In [11]:
cgm_df

GlucoseValue     PPGR
RegistrationCode ConnectionID Date                                            
111527           1926         2017-11-06 14:16:00+02:00          79.0  109.750
                              2017-11-06 14:31:00+02:00          83.0  132.500
                              2017-11-06 14:46:00+02:00          86.0  153.500
                              2017-11-06 15:01:00+02:00          89.0  170.750
                              2017-11-06 15:16:00+02:00         118.0  127.375
...                                                               ...      ...
997735           5832         2019-07-21 07:45:00+03:00         117.0      NaN
                              2019-07-21 08:00:00+03:00         119.0      NaN
                              2019-07-21 08:15:00+03:00         127.0      NaN
                              2019-07-21 08:30:00+03:00         138.0      NaN
                              2019-07-21 08:45:00+03:00         135.0      NaN

[3478385 rows x 2 columns]

In [16]:
cgm_df.reset_index(inplace=True)

In [17]:
cgm_df['Day'] = cgm_df['Date'].dt.date
cgm_df

/usr/wisdom/python3.5.3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


RegistrationCode ConnectionID                      Date  GlucoseValue  \
0                 111527         1926 2017-11-06 14:16:00+02:00          79.0   
1                 111527         1926 2017-11-06 14:31:00+02:00          83.0   
2                 111527         1926 2017-11-06 14:46:00+02:00          86.0   
3                 111527         1926 2017-11-06 15:01:00+02:00          89.0   
4                 111527         1926 2017-11-06 15:16:00+02:00         118.0   
...                  ...          ...                       ...           ...   
3478380           997735         5832 2019-07-21 07:45:00+03:00         117.0   
3478381           997735         5832 2019-07-21 08:00:00+03:00         119.0   
3478382           997735         5832 2019-07-21 08:15:00+03:00         127.0   
3478383           997735         5832 2019-07-21 08:30:00+03:00         138.0   
3478384           997735         5832 2019-07-21 08:45:00+03:00         135.0   

            PPGR         Day  
0        109.750  2017-11-06  
1        132.500  2017-11-06  
2        153.500  2017-11-06  
3        170.750  2017-11-06  
4        127.375  2017-11-06  
...          ...         ...  
3478380      NaN  2019-07-21  
3478381      NaN  2019-07-21  
3478382      NaN  2019-07-21  
3478383      NaN  2019-07-21  
3478384      NaN  2019-07-21  

[3478385 rows x 6 columns]

In [18]:
stats = cgm_df.copy()
#stats = drop(['Date', 'PPGR'], axis=1)

In [19]:
stats

RegistrationCode ConnectionID                      Date  GlucoseValue  \
0                 111527         1926 2017-11-06 14:16:00+02:00          79.0   
1                 111527         1926 2017-11-06 14:31:00+02:00          83.0   
2                 111527         1926 2017-11-06 14:46:00+02:00          86.0   
3                 111527         1926 2017-11-06 15:01:00+02:00          89.0   
4                 111527         1926 2017-11-06 15:16:00+02:00         118.0   
...                  ...          ...                       ...           ...   
3478380           997735         5832 2019-07-21 07:45:00+03:00         117.0   
3478381           997735         5832 2019-07-21 08:00:00+03:00         119.0   
3478382           997735         5832 2019-07-21 08:15:00+03:00         127.0   
3478383           997735         5832 2019-07-21 08:30:00+03:00         138.0   
3478384           997735         5832 2019-07-21 08:45:00+03:00         135.0   

            PPGR         Day  
0        109.750  2017-11-06  
1        132.500  2017-11-06  
2        153.500  2017-11-06  
3        170.750  2017-11-06  
4        127.375  2017-11-06  
...          ...         ...  
3478380      NaN  2019-07-21  
3478381      NaN  2019-07-21  
3478382      NaN  2019-07-21  
3478383      NaN  2019-07-21  
3478384      NaN  2019-07-21  

[3478385 rows x 6 columns]

In [22]:
# from scipy.stats import variation

In [23]:
# test = means.groupby(['RegistrationCode', 'day'], as_index=False).agg(
#                      {'GlucoseValue':['mean','std', variation]})

Seems like variation calculation lasts a bit too long. Better ways?

In [28]:
# %%time
# means.groupby(['RegistrationCode', 'day'], as_index=False).agg({'GlucoseValue':['mean', 'std', variation]})

In [29]:
# %%time
# means.groupby(['RegistrationCode', 'day'], as_index=False).agg({'GlucoseValue':['mean','std']})
# test['GlucoseValue']['CV'] = test['GlucoseValue']['std'] / test['GlucoseValue']['mean']
# test

The above method is much faster

In [22]:
stats = stats.groupby(['RegistrationCode', 'ConnectionID', 'Day'], as_index=False)['GlucoseValue'].agg(['mean','std'])
stats['CV'] = stats['std'] / stats['mean']

In [24]:
stats

mean        std        CV
RegistrationCode ConnectionID Day                                        
111527           1926         2017-11-06  113.897436  38.182246  0.335234
                              2017-11-07  110.072165  21.003595  0.190817
                              2017-11-08  113.020833  19.443294  0.172033
                              2017-11-09  110.621053  22.600216  0.204303
                              2017-11-10  109.989583  22.246227  0.202258
...                                              ...        ...       ...
997735           5832         2019-07-17  108.750000  14.211930  0.130684
                              2019-07-18  112.376344  12.823140  0.114109
                              2019-07-19  109.163043  10.772653  0.098684
                              2019-07-20  114.458333  34.610286  0.302383
                              2019-07-21  111.428571   9.249120  0.083005

[43667 rows x 3 columns]

## MAGE calculation

I need the DataFrame containing three columns:

- date: date of the observation with format yyyy/mm/dd.

- time: time of the observation with 24 hour format hh:mm:ss.

- glucose: glucose value of the observation in [mg/dl].

In [32]:
cgm_df

RegistrationCode ConnectionID                      Date  GlucoseValue  \
0                 111527         1926 2017-11-06 14:16:00+02:00          79.0   
1                 111527         1926 2017-11-06 14:31:00+02:00          83.0   
2                 111527         1926 2017-11-06 14:46:00+02:00          86.0   
3                 111527         1926 2017-11-06 15:01:00+02:00          89.0   
4                 111527         1926 2017-11-06 15:16:00+02:00         118.0   
...                  ...          ...                       ...           ...   
3478380           997735         5832 2019-07-21 07:45:00+03:00         117.0   
3478381           997735         5832 2019-07-21 08:00:00+03:00         119.0   
3478382           997735         5832 2019-07-21 08:15:00+03:00         127.0   
3478383           997735         5832 2019-07-21 08:30:00+03:00         138.0   
3478384           997735         5832 2019-07-21 08:45:00+03:00         135.0   

            PPGR         day  
0        109.750  2017-11-06  
1        132.500  2017-11-06  
2        153.500  2017-11-06  
3        170.750  2017-11-06  
4        127.375  2017-11-06  
...          ...         ...  
3478380      NaN  2019-07-21  
3478381      NaN  2019-07-21  
3478382      NaN  2019-07-21  
3478383      NaN  2019-07-21  
3478384      NaN  2019-07-21  

[3478385 rows x 6 columns]

In [6]:
mage_df = cgm.df
mage_df

GlucoseValue     PPGR
RegistrationCode ConnectionID Date                                            
111527           1926         2017-11-06 14:16:00+02:00          79.0  109.750
                              2017-11-06 14:31:00+02:00          83.0  132.500
                              2017-11-06 14:46:00+02:00          86.0  153.500
                              2017-11-06 15:01:00+02:00          89.0  170.750
                              2017-11-06 15:16:00+02:00         118.0  127.375
...                                                               ...      ...
997735           5832         2019-07-21 07:45:00+03:00         117.0      NaN
                              2019-07-21 08:00:00+03:00         119.0      NaN
                              2019-07-21 08:15:00+03:00         127.0      NaN
                              2019-07-21 08:30:00+03:00         138.0      NaN
                              2019-07-21 08:45:00+03:00         135.0      NaN

[3478385 rows x 2 columns]

In [39]:
dt = "2019-06-10T14:05:00+05:30"
dt = pd.to_datetime(dt, errors='ignore')
transformed_date = dt.strftime("%Y/%m/%d")
print(transformed_date)
print(type(transformed_date))

2019/06/10
<class 'str'>


In [7]:
mage_df = mage_df.reset_index(['RegistrationCode', 'ConnectionID'])


In [8]:
mage_df['Day'] = mage_df.index.date

In [20]:
# mage_df['Day'] = pd.to_datetime(mage_df['Day'])

In [22]:
# mage_df.Day.apply(lambda x: x.strftime('%Y/%m/%d')).astype(str)

Date
2017-11-06 14:16:00+02:00    2017/11/06
2017-11-06 14:31:00+02:00    2017/11/06
2017-11-06 14:46:00+02:00    2017/11/06
2017-11-06 15:01:00+02:00    2017/11/06
2017-11-06 15:16:00+02:00    2017/11/06
                                ...    
2019-07-21 07:45:00+03:00    2019/07/21
2019-07-21 08:00:00+03:00    2019/07/21
2019-07-21 08:15:00+03:00    2019/07/21
2019-07-21 08:30:00+03:00    2019/07/21
2019-07-21 08:45:00+03:00    2019/07/21
Name: Day, Length: 3478385, dtype: object

In [9]:
mage_df['Day'] = mage_df.Day.apply(lambda x: x.strftime('%Y/%m/%d')).astype(str)

In [10]:
mage_df['Time'] = mage_df.index.time
mage_df

RegistrationCode ConnectionID  GlucoseValue  \
Date                                                                    
2017-11-06 14:16:00+02:00           111527         1926          79.0   
2017-11-06 14:31:00+02:00           111527         1926          83.0   
2017-11-06 14:46:00+02:00           111527         1926          86.0   
2017-11-06 15:01:00+02:00           111527         1926          89.0   
2017-11-06 15:16:00+02:00           111527         1926         118.0   
...                                    ...          ...           ...   
2019-07-21 07:45:00+03:00           997735         5832         117.0   
2019-07-21 08:00:00+03:00           997735         5832         119.0   
2019-07-21 08:15:00+03:00           997735         5832         127.0   
2019-07-21 08:30:00+03:00           997735         5832         138.0   
2019-07-21 08:45:00+03:00           997735         5832         135.0   

                              PPGR         Day      Time  
Date                                                      
2017-11-06 14:16:00+02:00  109.750  2017/11/06  14:16:00  
2017-11-06 14:31:00+02:00  132.500  2017/11/06  14:31:00  
2017-11-06 14:46:00+02:00  153.500  2017/11/06  14:46:00  
2017-11-06 15:01:00+02:00  170.750  2017/11/06  15:01:00  
2017-11-06 15:16:00+02:00  127.375  2017/11/06  15:16:00  
...                            ...         ...       ...  
2019-07-21 07:45:00+03:00      NaN  2019/07/21  07:45:00  
2019-07-21 08:00:00+03:00      NaN  2019/07/21  08:00:00  
2019-07-21 08:15:00+03:00      NaN  2019/07/21  08:15:00  
2019-07-21 08:30:00+03:00      NaN  2019/07/21  08:30:00  
2019-07-21 08:45:00+03:00      NaN  2019/07/21  08:45:00  

[3478385 rows x 6 columns]

In [36]:
mage_df = mage_df.reset_index()

In [38]:
mage_df = mage_df.drop(['Date','PPGR'], axis=1)

RegistrationCode ConnectionID  GlucoseValue         Day      Time
0                 111527         1926          79.0  2017/11/06  14:16:00
1                 111527         1926          83.0  2017/11/06  14:31:00
2                 111527         1926          86.0  2017/11/06  14:46:00
3                 111527         1926          89.0  2017/11/06  15:01:00
4                 111527         1926         118.0  2017/11/06  15:16:00
...                  ...          ...           ...         ...       ...
3478380           997735         5832         117.0  2019/07/21  07:45:00
3478381           997735         5832         119.0  2019/07/21  08:00:00
3478382           997735         5832         127.0  2019/07/21  08:15:00
3478383           997735         5832         138.0  2019/07/21  08:30:00
3478384           997735         5832         135.0  2019/07/21  08:45:00

[3478385 rows x 5 columns]

In [42]:
cols = mage_df.columns.tolist()
cols

['RegistrationCode', 'ConnectionID', 'GlucoseValue', 'Day', 'Time']

In [45]:
cols = ['RegistrationCode', 'ConnectionID', 'Day', 'Time', 'GlucoseValue']

In [46]:
mage_df = mage_df[cols]

In [47]:
mage_df

RegistrationCode ConnectionID         Day      Time  GlucoseValue
0                 111527         1926  2017/11/06  14:16:00          79.0
1                 111527         1926  2017/11/06  14:31:00          83.0
2                 111527         1926  2017/11/06  14:46:00          86.0
3                 111527         1926  2017/11/06  15:01:00          89.0
4                 111527         1926  2017/11/06  15:16:00         118.0
...                  ...          ...         ...       ...           ...
3478380           997735         5832  2019/07/21  07:45:00         117.0
3478381           997735         5832  2019/07/21  08:00:00         119.0
3478382           997735         5832  2019/07/21  08:15:00         127.0
3478383           997735         5832  2019/07/21  08:30:00         138.0
3478384           997735         5832  2019/07/21  08:45:00         135.0

[3478385 rows x 5 columns]

## Heatmap

In [25]:
stats

mean        std        CV
RegistrationCode ConnectionID Day                                        
111527           1926         2017-11-06  113.897436  38.182246  0.335234
                              2017-11-07  110.072165  21.003595  0.190817
                              2017-11-08  113.020833  19.443294  0.172033
                              2017-11-09  110.621053  22.600216  0.204303
                              2017-11-10  109.989583  22.246227  0.202258
...                                              ...        ...       ...
997735           5832         2019-07-17  108.750000  14.211930  0.130684
                              2019-07-18  112.376344  12.823140  0.114109
                              2019-07-19  109.163043  10.772653  0.098684
                              2019-07-20  114.458333  34.610286  0.302383
                              2019-07-21  111.428571   9.249120  0.083005

[43667 rows x 3 columns]

In [28]:
import plotly

ImportError: No module named 'plotly'